In [ ]:
import signal

class TimeOut:
    """force timing out for things that get stuck"""
    def __init__(self, seconds=1, error_message='Timeout'):
        self.seconds = seconds
        self.error_message = error_message

    def handle_timeout(self, signum, frame):
        """handle timeout"""
        raise TimeoutError(self.error_message)

    def __enter__(self):
        signal.signal(signal.SIGALRM, self.handle_timeout)
        signal.alarm(self.seconds)

    def __exit__(self, type, value, traceback):
        signal.alarm(0)

# Extract pole-like street furniture

In this notebook we demonstrate how we can extract pole-like objects form labeled point clouds.

In this code we assume the point clouds have been labelled following the process in our [Urban PointCloud Processing](https://github.com/Amsterdam-AI-Team/Urban_PointCloud_Processing/tree/main/datasets) project. For more information on the specifics of the datasets used, see [the description there](https://github.com/Amsterdam-AI-Team/Urban_PointCloud_Processing/blob/main/datasets/README.md).

In [ ]:
# Add project src to path.
import set_path

import numpy as np
import pandas as pd
import geopandas as gpd
import os
import pathlib
import laspy
import uuid
from tqdm import tqdm

#from upcp.labels import Labels 
from labels import Labels
from upcp.utils import las_utils
from upcp.utils import ahn_utils
import upcp.utils.bgt_utils as bgt_utils
from upc_analysis import PoleExtractor

In [ ]:
# Select location
#my_location = '201001' # Oost
#my_location = '200923' # Zuidoost
#my_location = '200921' # Centrum
#my_location = '200920' # Nieuw-West
#my_location = '200918' # Noord
my_location = '200903' # Zuid
#my_location = '200824' # West
#my_location = '200823' # Haven

# We provide some example data for demonstration purposes.
base_folder = '/home/azureuser/cloudfiles/code/blobfuse/ovl'
dataset_folder = f'{base_folder}/pointcloud/Unlabeled/Amsterdam/nl-amsd-{my_location}-7415-laz/las_processor_bundled_out/'  # folder with point clouds
pred_folder = f'{base_folder}/predictions/nl-amsd-{my_location}-7415-laz/'  # folder with predictions as npz files
prefix = 'filtered_'
prefix_pred = 'pred_'
files = list(pathlib.Path(dataset_folder).glob(f'{prefix}*.laz'))

# Only predictions were made for files bigger than 10kb
files = [f for f in files if os.path.getsize(f) > 10000]

# Define the class we are interested in
target_label = 60

# Labels used for ground points ('Road' and 'Other ground')
ground_labels = [1, 9]

# AHN data reader for elevation data. This is optional: the data is only used when
# the ground elevation cannot be determined from the labeled point cloud itself.
ahn_data_folder = f'{base_folder}/ahn/Amsterdam/ahn4_npz/'
ahn_reader = ahn_utils.NPZReader(ahn_data_folder, caching=False)

# BGT data reader for building shapes. This is optional: the data is used to check
# whether an extracted object is located within a building footprint. This might
# indicate a false positive.
# If an appropriate dataset is available, uncomment these lines to use it:
if os.path.exists(f'{base_folder}/bgt/bgt_buildings_oost.csv'):
    bgt_building_file = f'{base_folder}/bgt/bgt_buildings_oost.csv'
    bld_reader = bgt_utils.BGTPolyReader(bgt_building_file)
else:
    bld_reader = None
    print('no building file')

# Trees data for filtering. This is optional: the data is used to check
# whether an extracted object is located very close to a tree. This might
# indicate a false positive.
filename_trees = f'{base_folder}/trees/obj_vgo_boom_view.gpkg'

# Settings for noise filtering
EPS_N = 0.1
MIN_SAMPLES_N = 50

# Settings for clustering
EPS = 0.1
MIN_SAMPLES = 150

In [ ]:
output_file = f'{base_folder}/poles_extracted_{my_location}.csv'
output_file_filter = f'{base_folder}/poles_extracted_{my_location}_filtered.csv'

# Define and create out folder for images of found clusters
img_out_folder = f'{base_folder}/images/{my_location}/'
if not os.path.isdir(img_out_folder):
    os.makedirs(img_out_folder + 'object_all_axes/')
    os.makedirs(img_out_folder + 'object_per_axis/x/')
    os.makedirs(img_out_folder + 'object_per_axis/y/')

---
## Extracting pole-like objects

This method works by clustering points of a given target class, and then using statistics and PCA analysis on each cluster to determine the exact pole.

The result is a dataset with the following features for each extracted object:
```txt
rd_x, rd_y, z = X, Y, Z coordinates of the base of the pole
tx, ty, tz    = X, Y, Z coordinates of the top of the pole
height        = the height of the pole, in m
angle         = the angle of the pole, in degrees w.r.t. vertical
prob          = the average probability of the classification, if this data is available in the point cloud
n_points      = the number of points of the object
in_bld        = flag that indicates whether the object is located inside a building footprint
debug         = debug code, see below
tilecode      = tilecode in which the object was found
```
The debug code `A_B` indicates potential issues with either the ground elevation (A) or the pole extraction (B). A can be either 0 (no problems), 1 (no ground elevation found in the point cloud), or 2 (no ground elevation found in AHN). B can be either 0 (no problems), 3 (not enough data to determine the angle), or 4 (not enough data to determine the exact location).

In [ ]:
# Create PoleExtractor object
pole_extractor = PoleExtractor(target_label, ground_labels,
                               ahn_reader=ahn_reader, building_reader=bld_reader,
                               eps_noise=EPS_N, min_samples_noise=MIN_SAMPLES_N,
                               eps=EPS, min_samples=MIN_SAMPLES)   

In [ ]:
#files = files[:200]  # TODO remove

In [ ]:
# Loop over point cloud files and extract objects
locations = []
for file in tqdm(files):
    try:
        with TimeOut(240):
            tilecode = las_utils.get_tilecode_from_filename(file.as_posix())
            pc = laspy.read(file)
            npz_file = np.load(pred_folder + prefix_pred + tilecode + '.npz')
            labels = npz_file['label']
            if 'probability' in pc.point_format.extra_dimension_names:
                probabilities = pc.probability
            else:
                probabilities = np.zeros_like(labels)
            if np.count_nonzero(labels == target_label) > 0:
                points = np.vstack((pc.x, pc.y, pc.z)).T
                colors = np.vstack((pc.green, pc.red, pc.blue)).T
                tile_locations = pole_extractor.get_pole_locations(points, colors, labels, probabilities, tilecode)
                locations.extend([(*x, tilecode) for x in tile_locations])
    except:
        continue

HEADERS = ['rd_x', 'rd_y', 'z', 'tx', 'ty', 'tz', 'height', 'angle', 'm_r', 'm_g', 'm_b', 'radius','prob', 'n_points', 'in_bld', 'debug', 'tilecode']
poles_df = pd.DataFrame(locations, columns=HEADERS)

In [ ]:
print(len(poles_df))
poles_df.head(2)

#### Add unique identifier to each row

In [ ]:
uuids = [uuid.uuid4() for _ in range(len(poles_df.index))]
poles_df.insert(0, 'identifier', uuids)

#### Store unfiltered data

In [ ]:
poles_df.to_csv(output_file, index=False)

In [ ]:
poles_df = pd.read_csv(output_file) # TODO remove

### Filter out likely false positives

#### Remove too small or large poles

In [ ]:
poles_df = poles_df[poles_df['height'] > 1.8]
poles_df = poles_df[poles_df['height'] < 16]
len(poles_df)

#### Remove poles within buildings

In [ ]:
print(poles_df['in_bld'].value_counts())
#poles_df = poles_df[poles_df['in_bld'] == 0]  # TODO investigate whether we can apply this filter
#len(poles_df)

#### Remove 'poles' that are actually trees

In [ ]:
# Get trees data
df_trees = gpd.read_file(filename_trees)
df_trees_o = df_trees[df_trees['Stadsdeel of kern'] == 'Zuid']  # 'Zuid', 'Oost', 'Zuidoost', 'Nieuw-West', 'Noord', 'West', 'Centrum', 'Westpoort'
del df_trees
df_trees_o.shape

In [ ]:
# Add buffer around trees
df_trees_o['buffer'] = df_trees_o['geometry'].buffer(0.3)
df_trees_o = df_trees_o.set_geometry('buffer')

# Check whether pole is in buffered trees
poles_df = gpd.GeoDataFrame(poles_df, geometry=gpd.points_from_xy(poles_df.rd_x, poles_df.rd_y), crs='EPSG:28992')
gdf_sjoin = poles_df.sjoin(df_trees_o, predicate='within')

# Remove poles that are within buffered trees
poles_df = poles_df[~poles_df.index.isin(gdf_sjoin.index)]
len(poles_df)

#### Store filtered data

In [ ]:
# Save the data in CSV format.
poles_df = poles_df.reset_index()
poles_df.to_csv(output_file_filter, index=False)